In [2]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [3]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [4]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [5]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [6]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [14]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [22]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.552s

OK


In [7]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [8]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a47304402207db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11022010178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67feffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600


In [17]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))


print("-----")
print("Mine")
passphrase = b'mynima@protonmail.comBorderTerriers.Becausepeoplearestupid.'
secret = little_endian_to_int(hash256(passphrase))
private = PrivateKey(secret)
public = private.point.address(compressed=True, testnet=True)
print("Public: " + public)
print("-----")

mn81594PzKZa9K3Jyy1ushpuEzrnTnxhVg
-----
Mine
Public: n1Eqn2vTXR4xfy7UVJGXnGn2DxQpqWXiWH
-----


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [13]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))


.
----------------------------------------------------------------------
Ran 1 test in 0.407s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [48]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

utxo_in = 0.002
btc_to_sats = 100000000
fee_factor = 0.002
fee_set = 230

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
# target amount should be 60% of the output amount
target_amount = int((utxo_in*btc_to_sats)*0.1)

# set the fee to some reasonable amount
if fee_set == 0:
    fee = int((utxo_in*btc_to_sats)*fee_factor)
else:
    fee = fee_set

# get the private key from the exercise in Chapter 4
# change address should be the address generated from Chapter 4
change_address = 'n1Eqn2vTXR4xfy7UVJGXnGn2DxQpqWXiWH'

# change amount = amount from the prev tx - target amount - fee
change_amount = int((utxo_in*btc_to_sats)) - target_amount - fee


# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
prev_tx = bytes.fromhex("5e001a72fdbbc09961ed2aeb02d02da8a6b7d3543ed96d1243bba9722599a51f")
prev_index = 1
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence
tx_ins = []
tx_in = TxIn(prev_tx, prev_index)
tx_ins.append(tx_in)


tx_outs = []
# create a transaction output for the target amount and address
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_outs.append(target_output)
# create a transaction output for the change amount and address
change_h160 = decode_base58(change_address)
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
tx_outs.append(change_output)


# create the transaction object
tx_obj = Tx(1, tx_ins, tx_outs, 0, True)

# sign the one input in the transaction object using the private key
passphrase = b'mynima@protonmail.comBorderTerriers.Becausepeoplearestupid.'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)

print("-----")
print("Send: " + str(target_amount))
print("Fee: " + str(fee))
print("Change: " + str(change_amount))

print("-----")
print("Signed Transaction(s):")
for i, tx_in in enumerate(tx_ins):
    print(tx_obj.sign_input(i-1, private_key))

# print the transaction's serialization in hex
print("-----")
print("TXN:")
print(tx_obj.serialize().hex())

# broadcast at http://testnet.blockchain.info/pushtx

-----
Send: 20000
Fee: 230
Change: 179770
-----
Signed Transaction:
True
-----
TXN:
01000000011fa5992572a9bb43126dd93e54d3b7a6a82dd002eb2aed6199c0bbfd721a005e010000006a473044022055e23928eab25b7bd8eaef9b121005d462ecf8d43730ec865ff261322affe5ca022063cc0183f4f4972ddccc69875b0ab8b68705b545e059e7a4d247b4db114aa48b0121030330b5abffcf0d488c878a37ac6ac705c7a92041898be4b725a19128fe2ca9bdffffffff02204e0000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac3abe0200000000001976a914d8566b7013b70035913951894b37ea21c9352cd088ac00000000


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [62]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'n1Eqn2vTXR4xfy7UVJGXnGn2DxQpqWXiWH'

# get the private key from the exercise in Chapter 4
passphrase = b'mynima@protonmail.comBorderTerriers.Becausepeoplearestupid.'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)

utxo_in = 0.0022977
btc_to_sats = 100000000
fee_set = 400

tx_ins = []
# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
prev_tx1 = bytes.fromhex("ebc428e30d10eb962506f3578e26a261a57600bb0cc3bba948f582628cce17e0")
prev_index1 = 1
# create the first transaction input with the default ScriptSig and
# sequence
tx_in1 = TxIn(prev_tx1, prev_index1)
tx_ins.append(tx_in1)

# get the prev_tx and prev_index from the transaction in Exercise 4
# create the second transaction input with the default ScriptSig and
# sequence
prev_tx2 = bytes.fromhex("0b6436950e31a90756ee2b1f5da59ee89b1c2b07aaeb0c38ff7c1a31f6547daa")
prev_index2 = 0
tx_in2 = TxIn(prev_tx2, prev_index2)
tx_ins.append(tx_in2)

# set the fee to some reasonable amount
# target amount should be the sum of the inputs - fee
fee = fee_set
target_amount = int((utxo_in-(fee/btc_to_sats))*btc_to_sats)
change_amount = int((utxo_in*btc_to_sats)) - target_amount - fee

# create a transaction output for the amount and address

# sign the first input using the private key
# sign the second input using the private key 
# print the transaction's serialization in hex

tx_outs = []
# create a transaction output for the target amount and address
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_outs.append(target_output)

# create the transaction object
tx_obj = Tx(1, tx_ins, tx_outs, 0, True)


print("-----")
print("Send: " + str(target_amount))
print("Fee: " + str(fee))
print("Change: " + str(change_amount))

print("-----")
print("Signed Transaction(s):")

print(tx_obj.sign_input(0, private_key))
print(tx_obj.sign_input(1, private_key))

# print the transaction's serialization in hex
print("-----")
print("TXN:")
print(tx_obj.serialize().hex())


# broadcast at http://testnet.blockchain.info/pushtx

-----
Send: 229370
Fee: 400
Change: 0
-----
Signed Transaction(s):
True
True
-----
TXN:
0100000002e017ce8c6282f548a9bbc30cbb0076a561a2268e57f3062596eb100de328c4eb010000006a47304402202343d1b8faca54ae4634d4ccfa5781322526ee4b7b1294c9fd9451f95057369702204d4e8eb92d9a21224e02d0d4560e34f74fff9ff3534cc1f33c802bb8b40336d90121030330b5abffcf0d488c878a37ac6ac705c7a92041898be4b725a19128fe2ca9bdffffffffaa7d54f6311a7cff380cebaa072b1c9be89ea55d1f2bee5607a9310e9536640b000000006b483045022100e4235308ef3edfa75ea2b5b4293746db22fc688610aaf91ee56d1adedc19bd41022063e61ed9e67ee0c77e8ef3dd610072c68f055723f9897e79b4235e2d0683d4d20121030330b5abffcf0d488c878a37ac6ac705c7a92041898be4b725a19128fe2ca9bdffffffff01fa7f0300000000001976a914d8566b7013b70035913951894b37ea21c9352cd088ac00000000
